In [ ]:
%sudo apt-get install g++ openjdk-8-jdk python3-dev python3-pip curl
%python3 -m pip install konlpy
%pip install mecab-python3
%bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [1]:
from time import sleep
from konlpy.tag import Kkma, Okt, Mecab
tagger = Mecab()
# Look for NNG (보통 명사) '지진'
sampleText = "전쟁...기후 변화...지진 모든 것이 성경에 새겨진 종말의 징조다 그때가 오면 악마들이 온다 ........... 악마는 예전에도 지금도 미래에도 있을 것인데 우리가 지금 악마의 시대에 살고 있다"
if "지진/NNG" in tagger.pos(sampleText, join=True):
    print("yes")
else:
    print("no")

yes


In [14]:
# 화재/NNG
# 싱크홀/NNP
tagger.pos("방지턱 없는 데 차들 ‘덜컹’…연희동 싱크홀 10분 전 상황(영상) 서울 서대문구 연희동 4차선 도로에서 싱크홀(땅꺼짐 현상)이 발생해 SUV 차량에 타고 있던 운전자 등 2명이 부상을 당한 가운데 사고가...")


[('방지', 'NNG'),
 ('턱', 'MAG'),
 ('없', 'VA'),
 ('는', 'ETM'),
 ('데', 'NNB'),
 ('차', 'NNG'),
 ('들', 'XSN'),
 ('‘', 'SY'),
 ('덜컹', 'MAG'),
 ('’…', 'SY'),
 ('연희동', 'NNP'),
 ('싱크홀', 'NNP'),
 ('10', 'SN'),
 ('분', 'NNBC'),
 ('전', 'NNG'),
 ('상황', 'NNG'),
 ('(', 'SSO'),
 ('영상', 'NNG'),
 (')', 'SSC'),
 ('서울', 'NNP'),
 ('서대문구', 'NNP'),
 ('연희동', 'NNP'),
 ('4', 'SN'),
 ('차선', 'NNG'),
 ('도로', 'NNG'),
 ('에서', 'JKB'),
 ('싱크홀', 'NNP'),
 ('(', 'SSO'),
 ('땅', 'NNG'),
 ('꺼짐', 'VV+EC+VX+ETN'),
 ('현상', 'NNG'),
 (')', 'SSC'),
 ('이', 'JKS'),
 ('발생', 'NNG'),
 ('해', 'XSV+EC'),
 ('SUV', 'SL'),
 ('차량', 'NNG'),
 ('에', 'JKB'),
 ('타', 'VV'),
 ('고', 'EC'),
 ('있', 'VX'),
 ('던', 'ETM'),
 ('운전자', 'NNG'),
 ('등', 'NNB'),
 ('2', 'SN'),
 ('명', 'NNBC'),
 ('이', 'JKS'),
 ('부상', 'NNG'),
 ('을', 'JKO'),
 ('당한', 'VV+ETM'),
 ('가운데', 'NNG'),
 ('사고', 'NNG'),
 ('가', 'JKS'),
 ('.', 'SF'),
 ('..', 'SY')]

In [3]:
from dataclasses import dataclass
from typing import Literal, Optional
import numpy
import pandas

@dataclass
class Post:
    gallery: str
    title: str
    content: str
    date: numpy.datetime64
    keyword: str
    searchMode: Literal["accuracy", "latest"]
    disaster: Optional[bool]

In [7]:
df = pandas.read_csv("./data_dcinside/data_deduped.csv", index_col=0, dtype={"content": str, "title": str})

In [8]:
df.dtypes

gallery        object
title          object
content        object
date           object
keyword        object
searchMode     object
disaster      float64
dtype: object

In [15]:
keywordToTag = {
    "지진": "지진/NNG",
    "화재": "화재/NNG",
    "싱크홀": "싱크홀/NNP"
}

def crudeLabel(row):
    tag = keywordToTag.get(row.keyword)
    joinedContent = str(row.title) + '\n' + str(row.content)
    if tag:
        if tag in tagger.pos(joinedContent, join=True):
            return True
        else:
            return False
    return None

labels = list(map(crudeLabel, df.itertuples()))
df["disaster"] = labels

In [16]:
df

,gallery,title,content,date,keyword,searchMode,disaster
0,지진,2024년 12월 6일 미국지진 분석,CA 깊이0.6km Ⅷ 쓰나미경보 ChatGPT의 말: 2024년 12월 5일 캘리...,2024-12-06 04:48:00,지진,accuracy,True
1,지진,2024년 11월 5일~2024년 12월 5일 30일간 미국지진 분석,↓2024-11-05~2024-12-05 미국지진 M3.7이상 ★ 오타로 인해 자료...,2024-12-05 16:21:00,지진,accuracy,True
2,러시아-우크라이나,"쿠바, 허리케인 끝나자 이번엔 규모 6.8 지진…""이런 지진은 처음""",지진 후 15회 이상 여진도 발생…쓰나미 위험은 없어 사망자 아직 보고되지 않았지만...,2024-11-11 08:37:00,지진,accuracy,True
3,일본 지진,11월17일~12월 5일 일본지진 Chatgpt분석,★ 오타로 인해 자료에 오류가 있을 가능성 있습니다. 2024-11-17 21:16...,2024-12-05 15:46:00,지진,accuracy,True
5,지진,님들아 제주도 지진 몆주간 계속 났는데 큰 지진 나나요?,옛날에 자잘한 지진 계속 나면 나중에 큰 지진 난다고 들었는데 지금 제주도 9월부터...,2024-11-14 00:14:00,지진,accuracy,True
...,...,...,...,...,...,...,...
1242,메르스,띠닌 근데 애미가 말 무례하게 하면,발로 배 걷어차고 뺨 ㅈㄴ 때림 이년도 그냥 애비랑 똑같은 한남(후천적 남근)이라 ...,2024-12-09 15:34:00,근데,latest,None
1243,림월드,근데 시나리오 추가된 건축물에는 왜 하필 일반 전선이 기본적으로 들어감?,SOS도 엔진에 일반 전선 붙어있어서 거기서 전기 터진 적도 있고 지금 데드맨 스위...,2024-12-09 15:34:00,근데,latest,None
1244,원신 project,근데 차스카 분산 때문에 의외로 딜 박힘,마지막에 발사되는게 첫발인 바람탄이라 확산이 확정적으로 일어나서 적들 한군데 몰려있...,2024-12-09 15:34:00,근데,latest,None
1245,걸스플래닛999,근데에스파 왤케 인기 많노,음악 걍 그런데 베몬이나 있지가 더 괜찮은디 - dc official App,2024-12-09 15:34:00,근데,latest,None


In [17]:
df.to_csv("./data_dcinside/data_crude_label.csv")